In [14]:
import numpy as np
import pandas as pd
from pandas.api.types import is_string_dtype, is_object_dtype
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from cvxopt import matrix, solvers

In [15]:
# df = pd.read_csv('Datasets/bank-customer-churn-prediction.csv')
# df = pd.read_csv('Datasets/financial-risk-for-loan-approval.csv')
# df = pd.read_csv('Datasets/loan-approval-classification-dataset.csv')

df = pd.read_csv('Datasets/campus-placement-prediction.csv')
# df = pd.read_csv('Datasets/predict-dropout-or-academic-success.csv')
# df = pd.read_csv('Datasets/student-performance-dataset.csv')

# df = pd.read_csv('Datasets/fetal-health-classifiation.csv')
# df = pd.read_csv('Datasets/heart-disease-health-indicators-dataset.csv')
# df = pd.read_csv('Datasets/patient-treatment-classification.csv')

In [16]:
def corr_matrix(dataframe):
    preprocessing(dataframe)
    correlation_matrix = dataframe.corr()
    plt.figure(figsize=(10, 5))
    sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
    plt.show()

In [26]:
class DBSCAN:
    def __init__(self, eps=1.0, minPts=5, equation='euclidean'):
        self.eps = eps # Radius for neighbourhood search
        self.minPts = minPts # Minimum number of points to form a cluster
        self.equation = equation
        
    def fit(self, dataframe):
        # Initialise labels array with -1
        self.labels = np.zeros(len(dataframe), dtype=int) - 1
        self.clusters = [] # List to hold all clusters
        cidx = 0 # Cluster ID counter
        for x in range(len(dataframe)):
            if self.labels[x] == -1:
                neighbours = self.get_neighbours(dataframe, x)
                if len(neighbours) < self.minPts:
                    self.labels[x] = 0 # Mark point as anomaly
                else:
                    cidx += 1
                    self.clusters.append([x])
                    self.labels[x] = cidx # Assign cluster ID to the point

                    # Exapnd the cluster with points
                    for y in neighbours:
                        if self.labels[y] == -1:
                            self.labels[y] = cidx
                            self.clusters[cidx-1].append(y)
                            neighbours2 = self.get_neighbours(dataframe, y)
                            if len(neighbours2) >= self.minPts:
                                neighbours += list(set(neighbours2) - set(neighbours))
                        elif self.labels[y] == 0:
                            self.labels[y] = cidx
                            self.clusters[cidx-1].append(y)
        return self.clusters

    def get_neighbours(self, dataframe, x):
        neighbours = set()
        # Iterate over all points to calculate distances
        for y in range(len(dataframe)):
            if self.distance(dataframe[x], dataframe[y]) <= self.eps: # Check if within eps distance
                neighbours.add(y)
        return list(neighbours)
    
    def distance(self, x, y):
        # Euclidean distance
        return np.sqrt(np.sum((x-y)**2))
                           

    def plot(self, data):
        # Create a 2D plot (instead of 3D)
        plt.figure(figsize=(10, 8))

        # Find noise points and clusters
        noise_idx = np.where(self.labels == 0)[0]
        cluster_idxs = [np.array(c) for c in self.clusters]

        # Assign colours to clusters
        colours = cm.rainbow(np.linspace(0, 1, len(self.clusters) + 1))

        # Plot each cluster
        for i, cluster_idx in enumerate(cluster_idxs):
            plt.scatter(data[cluster_idx, 0], data[cluster_idx, 1], 
                        color=colours[i], s=10, label=f'Cluster {i + 1}')

        # Plot anomalies
        if len(noise_idx) > 0:
            plt.scatter(data[noise_idx, 0], data[noise_idx, 1], 
                        color='black', s=10, label='Anomalies')

        # Add labels and legend
        plt.title("DBSCAN Clustering")
        plt.xlabel("Feature 1")
        plt.ylabel("Feature 2")
        plt.legend()
        plt.show()

In [27]:
class OCSVM:
    def __init__(self, nu=0.1, kernel='rbf', gamma=1.0):
        self.nu = nu
        self.kernel = kernel
        self.gamma = gamma

    def _kernel_function(self, X, Y=None):
        # Computer the kernel matrix
        if Y is None:
            Y = X
        if self.kernel == 'rbf':
            # Radial basis function
            # Approximate RBF kernel using Random Fourier Features
            n_features = 100  # Dimensionality of approximation
            rng = np.random.default_rng(seed=42)
            omega = rng.normal(0, np.sqrt(2 * self.gamma), size=(X.shape[1], n_features))
            b = rng.uniform(0, 2 * np.pi, size=n_features)
            X_proj = np.cos(X @ omega + b)
            Y_proj = np.cos(Y @ omega + b) if Y is not X else X_proj
            return X_proj @ Y_proj.T
        elif self.kernel == 'linear':
            # Linear kernel
            return np.dot(X, Y.T)
        elif self.kernel == 'poly':
            # Polynomial kernel
            return (np.dot(X, Y.T) + 1) ** 3
        else:
            raise ValueError("Unsupported kernel type")

    def fit(self, X):
        n_samples = X.shape[0]
        K = self._kernel_function(X)

        # Define the quadratic programming problem
        P = matrix(K)
        q = matrix(-np.ones((n_samples, 1)))
        
        G = matrix(np.vstack((-np.eye(n_samples), np.eye(n_samples))))
        h = matrix(np.hstack((np.zeros(n_samples), np.ones(n_samples) / (self.nu * n_samples))))
        
        A = matrix(1.0, (1, n_samples)) # Equaltiy constraint
        b = matrix(1.0)

        # Solve quadratic programming problem
        sol = solvers.qp(P, q, G, h, A, b)

        # Extract Lagrange multipliers (alphas)
        alphas = np.array(sol['x']).flatten()
        support_vector_indices = alphas > 1e-5
        self.alphas = alphas[support_vector_indices]
        self.support_vectors = X[support_vector_indices]
        self.K_sv = K[support_vector_indices][:, support_vector_indices]
        
        # Compute the intercept term
        self.rho = np.mean(self.K_sv @ self.alphas)

    def decision_function(self, X):
        # Compute the decision function
        K_test = self._kernel_function(X, self.support_vectors)
        return np.sum(K_test * self.alphas, axis=1) - self.rho

    def predict(self, X):
        # Predict whether the data is an inlier or an outlier
        return np.sign(self.decision_function(X))

    def plot(self, X):
        predictions = self.predict(X)
        inliers = X[predictions == 1]
        outliers = X[predictions == -1]

        # Print IDs of outliers
        outlier_ids = np.where(predictions == -1)[0]
        print("Outlier IDs:", outlier_ids)

        plt.figure(figsize=(8, 6))
        plt.scatter(inliers[:, 0], inliers[:, 1], c='blue', label='Inliers', s=20)
        plt.scatter(outliers[:, 0], outliers[:, 1], c='red', label='Outliers', s=20)
        plt.title('One-Class SVM Results')
        plt.xlabel('Feature 1')
        plt.ylabel('Feature 2')
        plt.legend()
        plt.grid(True)
        plt.show()

In [28]:
class IsolationTree:
    def __init__(self, max_depth):
        self.max_depth = max_depth
        self.split_feature = None
        self.split_value = None
        self.left = None
        self.right = None
        self.is_terminal = False
        self.size = 0

    def fit(self, X, depth=0):
        # Check the termination condition
        if depth >= self.max_depth or len(X) <= 1:
            self.is_terminal = True
            self.size = len(X)
            return

        # Randomly choose a feature and split value
        num_features = X.shape[1]
        self.split_feature = np.random.randint(0, num_features)
        min_val, max_val = np.min(X[:, self.split_feature]), np.max(X[:, self.split_feature])

        if min_val == max_val:  # No meaningful split
            self.is_terminal = True
            self.size = len(X)
            return

        self.split_value = np.random.uniform(min_val, max_val)

        # Partition the data
        left_mask = X[:, self.split_feature] < self.split_value # Mask for left subset
        right_mask = ~left_mask # Mask for right subset

        self.left = IsolationTree(self.max_depth)
        self.right = IsolationTree(self.max_depth)

        self.left.fit(X[left_mask], depth + 1)
        self.right.fit(X[right_mask], depth + 1)

    def path_length(self, X):
        path_lengths = np.zeros(X.shape[0])

        for i, x in enumerate(X):
            node = self
            depth = 0
            while not node.is_terminal: # Traverse tree until termianl node reached
                if x[node.split_feature] < node.split_value:
                    node = node.left
                else:
                    node = node.right
                depth += 1
            path_lengths[i] = depth + self._c(node.size)
        return path_lengths

    @staticmethod
    def _c(size):
        # Calculate average path length of a binary tree for given size
        if size <= 1:
            return 0
        return 2 * (np.log(size - 1) + 0.5772156649) - (2 * (size - 1) / size)


class IsolationForest:
    def __init__(self, n_estimators=100, max_samples=256, max_depth=None):
        self.n_estimators = n_estimators
        self.max_samples = max_samples
        self.max_depth = max_depth
        self.forest = []

    def fit(self, X):
        # Convert DataFrame to NumPy array if needed
        if isinstance(X, pd.DataFrame):
            X = X.values

        num_samples, num_features = X.shape
        self.max_samples = min(self.max_samples, num_samples)
        if self.max_depth is None:
            self.max_depth = int(np.ceil(np.log2(self.max_samples)))

        self.forest = []
        for _ in range(self.n_estimators):
            sample_indices = np.random.choice(num_samples, self.max_samples, replace=False)
            X_sample = X[sample_indices]
            tree = IsolationTree(self.max_depth)
            tree.fit(X_sample)
            self.forest.append(tree)

    def anomaly_score(self, X):
        # Convert DataFrame to NumPy array if needed
        if isinstance(X, pd.DataFrame):
            X = X.values

        path_lengths = np.zeros(X.shape[0])
        for tree in self.forest:
            path_lengths += tree.path_length(X)
        path_lengths /= len(self.forest)

        # Compute anomaly score based on path lengths
        c_n = 2 * (np.log(self.max_samples - 1) + 0.5772156649) - (2 * (self.max_samples - 1) / self.max_samples)
        scores = 2 ** (-path_lengths / c_n)
        return scores

    def predict(self, X, threshold=0.5):
        scores = self.anomaly_score(X)
        return np.where(scores > threshold, -1, 1)  # -1 for anomalies, 1 for normal

    def plot(self, X, predictions, feature_names=None):
        if isinstance(X, pd.DataFrame):
            X = X.values

        # Assign colors for anomalies and inliers
        colors = np.array(['blue' if p == 1 else 'red' for p in predictions])
        plt.figure(figsize=(10, 6))

        if X.shape[1] == 2:  # If data has 2 features
            plt.scatter(X[:, 0], X[:, 1], c=colors, alpha=0.8)
            plt.xlabel(feature_names[0])
            plt.ylabel(feature_names[1])
        else:  # For more than 2 features, plot only the first two
            plt.scatter(X[:, 0], X[:, 1], c=colors, alpha=0.8)
            plt.xlabel('Feature 1')
            plt.ylabel('Feature 2')
        plt.title('Isolation Forest - Anomaly Detection')
        plt.legend(handles=[plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, label='Inliers'),
                            plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='Anomalies')],
                   loc='upper right')
        plt.grid()
        plt.show()

In [29]:
# Function to identify unique ID column
def identify_unique_id(dataframe):
    continuous_ratio_threshold = 2
    unique_id_column = []
    for col in dataframe.columns:
        if (
            dataframe[col].nunique() == len(dataframe)  # Unique values equal to the number of rows
            and not pd.api.types.is_float_dtype(dataframe[col])  # Exclude float columns
        ):
            # Calculate range-to-unique ratio for integer columns
            if pd.api.types.is_integer_dtype(dataframe[col]):
                col_range = dataframe[col].max() - dataframe[col].min() + 1
                if col_range / dataframe[col].nunique() > continuous_ratio_threshold:
                    continue  # Likely a continuous integer column, not an ID
            unique_id_column = col
            print(unique_id_column)
    if unique_id_column:
        dataframe = dataframe.drop(columns=unique_id_column)
    return dataframe

In [21]:
def drop_OCSVM(dataframe):
    used_cols = [col for col in dataframe.columns if dataframe[col].nunique() > 50]
    print(used_cols)
    for i in range(len(used_cols)):
        for j in range(i+1, len(used_cols)):
            data = dataframe[[used_cols[i], used_cols[j]]]
            data_norm = StandardScaler().fit_transform(data)
            
            # Train custom One-Class SVM
            oc_svm = OCSVM(nu=0.01, kernel='rbf', gamma=0.2)
            oc_svm.fit(data_norm)
            oc_svm.plot(data_norm)
            outlier_ids = np.where(oc_svm.predict(data_norm) == -1)[0]
            dataframe = dataframe.drop(dataframe.index[outlier_ids]).reset_index(drop=True)
            print(len(dataframe))
            
            # Predict on training data_norm
            predictions = oc_svm.predict(data_norm)
    return dataframe

In [22]:
def drop_DBSCAN(dataframe):
    used_cols = [col for col in dataframe.columns if dataframe[col].nunique() > 50]
    print(used_cols)
    for i in range(len(used_cols)):
        for j in range(i+1, len(used_cols)):
            data = dataframe[[used_cols[i], used_cols[j]]]
            data_norm = StandardScaler().fit_transform(data)
    
            # Train custom DBSCAN
            dbscan = DBSCAN(eps=4, minPts=5, equation='euclidean')
            dbscan.fit(data_norm)
            dbscan.plot(data_norm)
            outlier_ids = np.where(dbscan.labels == 0)[0]
            dataframe = dataframe.drop(dataframe.index[outlier_ids]).reset_index(drop=True)
            print(len(dataframe))
    return dataframe

In [23]:
def drop_IF(dataframe):
    used_cols = [col for col in dataframe.columns if dataframe[col].nunique() > 50]
    print(used_cols)
    for i in range(len(used_cols)):
        for j in range(i+1, len(used_cols)):
            data = dataframe[[used_cols[i], used_cols[j]]]

            # Train Isolation Forest algorithm
            iforest = IsolationForest(n_estimators=100, max_samples=len(data), max_depth=100)
            iforest.fit(data)
            predictions = iforest.predict(data, threshold=0.73)
            iforest.plot(data, predictions, feature_names=['Absences', 'GPA'])
            
            outlier_ids = np.where(predictions == -1)[0]
            dataframe = dataframe.drop(dataframe.index[outlier_ids]).reset_index(drop=True)
            print(len(dataframe))
    return dataframe

In [24]:
def preprocessing(dataframe):
    dataframe = identify_unique_id(dataframe)
    dataframe = convert_dtype(dataframe)
    return dataframe

In [25]:
def convert_dtype(dataframe):
    for column in dataframe.columns:
        if is_object_dtype(dataframe[column]):
            unique_values = dataframe[column].unique()
            value_to_float = {value: float(index) for index, value in enumerate(unique_values)}
            dataframe[column] = dataframe[column].map(value_to_float)
        if is_string_dtype(dataframe[column]):
            unique_values = dataframe[column].unique()
            value_to_float = {value: float(index) for index, value in enumerate(unique_values)}
            dataframe[column] = dataframe[column].map(value_to_float)
    dataframe.info()
    return dataframe